In [2]:
!pip install langchain langchain-core langchain-google-genai pydantic flask pyngrok flask-cors


In [3]:
import os
import json
from typing import Optional, List, Dict, Any
import traceback

# Imports από LangChain και Pydantic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables.base import RunnableBinding

# Για τα Kaggle Secrets
from kaggle_secrets import UserSecretsClient

# Για Flask και ngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
import threading # Για να τρέχει το ngrok και το Flask ταυτόχρονα

print("Οι βιβλιοθήκες φορτώθηκαν επιτυχώς.")

Οι βιβλιοθήκες φορτώθηκαν επιτυχώς.


In [4]:
google_api_key = None
ngrok_authtoken = None

try:
    user_secrets = UserSecretsClient()
    google_api_key = user_secrets.get_secret("GOOGLE_API_KEY")
    print("Το Google API Key φορτώθηκε επιτυχώς από τα Kaggle Secrets.")

    ngrok_authtoken = user_secrets.get_secret("NGROK_AUTHTOKEN")
    print("Το Ngrok Authtoken φορτώθηκε επιτυχώς από τα Kaggle Secrets.")

except Exception as e:
    print(f"Σφάλμα κατά τη φόρτωση secrets: {e}")
    if not google_api_key:
        print("ΚΡΙΣΙΜΟ ΣΦΑΛΜΑ: Βεβαιωθείτε ότι έχετε προσθέσει το GOOGLE_API_KEY στα secrets.")
    if not ngrok_authtoken:
        print("ΚΡΙΣΙΜΟ ΣΦΑΛΜΑ: Βεβαιωθείτε ότι έχετε προσθέσει το NGROK_AUTHTOKEN στα secrets.")

# Ρύθμιση του ngrok authtoken
if ngrok_authtoken:
    conf.get_default().auth_token = ngrok_authtoken
    conf.get_default().monitor_thread = False # Απενεργοποίηση του monitor thread για απλότητα στο Kaggle

Το Google API Key φορτώθηκε επιτυχώς από τα Kaggle Secrets.
Το Ngrok Authtoken φορτώθηκε επιτυχώς από τα Kaggle Secrets.


In [7]:
database_content = {
  "theater_general_info": {
    "name": "Θέατρο 'Η Αυλαία'",
    "address": "Πλατεία Συντάγματος 1, 10563 Αθήνα",
    "phone": "210-3201234",
    "email": "info@ayleatheater.gr",
    "website": "www.ayleatheater.gr",
    "opening_hours_box_office": "Δευτέρα - Παρασκευή: 10:00 - 14:00 & 17:00 - 20:00, Σάββατο: 10:00 - 15:00. Τις ημέρες παραστάσεων, το ταμείο είναι ανοιχτό και μία ώρα πριν την έναρξη.",
    "parking": {
      "available": True,
      "details": "Διατίθεται περιορισμένος αριθμός δωρεάν θέσεων στάθμευσης στον υπόγειο χώρο του θεάτρου. Εναλλακτικά, υπάρχουν ιδιωτικοί χώροι στάθμευσης περιμετρικά της πλατείας Συντάγματος."
    },
    "accessibility": {
      "wheelchair_ramps": True,
      "accessible_seating": "Διαθέσιμες ειδικά διαμορφωμένες θέσεις για χρήστες αμαξιδίων. Παρακαλούμε ενημερώστε κατά την κράτηση.",
      "accessible_restrooms": True,
      "hearing_assistance": "Διατίθενται συστήματα ενίσχυσης ακοής κατόπιν αιτήματος.",
      "service_animals": "Τα ζώα υπηρεσίας είναι ευπρόσδεκτα στο θέατρο μας."
    },
    "location_notes": "Εύκολη πρόσβαση με Μετρό (Σταθμός Σύνταγμα) και όλα τα μέσα μαζικής μεταφοράς που εξυπηρετούν το κέντρο της Αθήνας."
  },
  "shows": [
    {
      "id": "show1",
      "title": "Άμλετ",
       "subtitle": "Κυρίως Παράσταση",
        "synopsis": "Το κλασικό δράμα του Σαίξπηρ για το διάσημο πρίγκιπα της Δανίας.",
        "duration": 120,
        "schedule": [
        {"date": "2025-11-15", "time": "18:00", "availability": 50, "price_eur": 15, "day_of_week": "Σάββατο"},
        {"date": "2025-11-16", "time": "18:00", "availability": 30, "price_eur": 15, "day_of_week": "Κυριακή"},
        {"date": "2025-11-22", "time": "18:00", "availability": 40, "price_eur": 15, "day_of_week": "Σάββατο"}
      ],
        "reviews": ["Εξαιρετική παράσταση! Τα παιδιά μας τη λάτρεψαν.", "Μια μαγευτική εμπειρία για μικρούς και μεγάλους."]
    },
    {
      "id": "show2",
     "title": "Ρωμαίος και Ιουλιέτα",
    "subtitle": "Κυρίως Παράσταση",
    "duration": 120,
    "description": "Η τραγική ιστορία των δύο νέων εραστών από τον Σαίξπηρ, ένα έργο γεμάτο πάθος, αντιπαραθέσεις οικογενειών και το κλασικό δίλημμα του έρωτα ενάντια στον καθιερωμένο νόμο.",
      "schedule": [
        {"date": "2025-11-14", "time": "21:00", "availability": 20, "price_eur": 20, "day_of_week": "Παρασκευή"},
        {"date": "2025-11-15", "time": "21:00", "availability": 10, "price_eur": 25, "day_of_week": "Σάββατο"},
        {"date": "2025-11-21", "time": "21:00", "availability": 15, "price_eur": 20, "day_of_week": "Παρασκευή"}
      ],
      "reviews": ["Απίστευτο γέλιο! Μια πολύ έξυπνη διασκευή που δεν πρέπει να χάσετε.", "Σύγχρονη και εύστοχη, μια από τις καλύτερες κωμωδίες της χρονιάς."]
    }
  ]
}

try:
    with open("database.json", 'w', encoding='utf-8') as f:
        json.dump(database_content, f, ensure_ascii=False, indent=2)
    print("Το αρχείο 'database.json' δημιουργήθηκε επιτυχώς.")
except Exception as e:
    print(f"Σφάλμα κατά τη δημιουργία του 'database.json': {e}")

Το αρχείο 'database.json' δημιουργήθηκε επιτυχώς.


In [8]:
# Συνάρτηση φόρτωσης δεδομένων
def load_theater_data(filepath="database.json") -> Dict[str, Any]:
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"ΣΦΑΛΜΑ NOTEBOOK: Το αρχείο δεδομένων '{filepath}' δεν βρέθηκε.")
        return {"theater_general_info": {}, "shows": []}
    except json.JSONDecodeError:
        print(f"ΣΦΑΛΜΑ NOTEBOOK: Το αρχείο '{filepath}' δεν είναι έγκυρο JSON.")
        return {"theater_general_info": {}, "shows": []}
    except Exception as e:
        print(f"ΣΦΑΛΜΑ NOTEBOOK κατά τη φόρτωση των δεδομένων του θεάτρου: {e}")
        return {"theater_general_info": {}, "shows": []}

# Pydantic Models (Εργαλεία) με Ελληνικές Περιγραφές
class GetShowInfoParameters(BaseModel):
    """Χρησιμοποίησε αυτό το εργαλείο όταν ο χρήστης ζητά πληροφορίες για μια συγκεκριμένη παράσταση ή για τις διαθέσιμες παραστάσεις γενικά."""
    show_name: Optional[str] = Field(None, description="Ο τίτλος της παράστασης για την οποία ρωτά ο χρήστης. Αν ο χρήστης ρωτά γενικά για 'παραστάσεις', 'τι παίζει', ή 'πρόγραμμα', αυτό μπορεί να είναι κενό.")
    date_query: Optional[str] = Field(None, description="Οποιαδήποτε ημερομηνία ή ημέρα αναφέρει ο χρήστης (π.χ., 'σήμερα', 'αύριο', 'Σάββατο', 'επόμενη εβδομάδα', '15/11/2025').")
    genre_query: Optional[str] = Field(None, description="Οποιοδήποτε είδος (genre) αναφέρει ο χρήστης (π.χ., 'κωμωδία', 'παιδική παράσταση').")

class BookTicketParameters(BaseModel):
    """Χρησιμοποίησε αυτό το εργαλείο όταν ο χρήστης δηλώνει ρητά ότι θέλει να ΚΛΕΙΣΕΙ εισιτήρια για μια συγκεκριμένη παράσταση."""
    first_name: Optional[str] = Field(None, description="Το όνομα του ατόμου που κάνει την κράτηση.")
    last_name: Optional[str] = Field(None, description="Το επώνυμο του ατόμου που κάνει την κράτηση.")
    show_name: str = Field(..., description="Ο ακριβής τίτλος της παράστασης για την οποία θα γίνει η κράτηση. Πρέπει να αντιστοιχεί σε έναν από τους τίτλους στα 'Διαθέσιμα Δεδομένα Παραστάσεων'.")
    num_tickets: int = Field(..., description="Ο αριθμός των εισιτηρίων που θέλει να κλείσει ο χρήστης.")
    date: str = Field(..., description="Η συγκεκριμένη ημερομηνία για την κράτηση (π.χ., '15/11/2025'). Πρέπει να είναι έγκυρη ημερομηνία από το πρόγραμμα της παράστασης.")
    time: Optional[str] = Field(None, description="Η συγκεκριμένη ώρα για την κράτηση (π.χ., '18:00', '21:00'). Πρέπει να είναι έγκυρη ώρα για την επιλεγμένη ημερομηνία από το πρόγραμμα της παράστασης.")

class GetTheaterInfoParameters(BaseModel):
    """Χρησιμοποίησε αυτό το εργαλείο όταν ο χρήστης ζητά γενικές πληροφορίες ΓΙΑ ΤΟ ΙΔΙΟ ΤΟ ΘΕΑΤΡΟ (π.χ. διεύθυνση, τηλέφωνο, πάρκινγκ, ιστοσελίδα, ώρες λειτουργίας, προσβασιμότητα)."""
    info_type: str = Field(..., description="Το συγκεκριμένο είδος πληροφορίας που ζητείται για το θέατρο (π.χ., 'διεύθυνση', 'τηλέφωνο', 'πάρκινγκ', 'προσβασιμότητα', 'τοποθεσία', 'επικοινωνία', 'γενικές_πληροφορίες').")

tools = [
    GetShowInfoParameters,
    BookTicketParameters,
    GetTheaterInfoParameters
]

def format_show_info_response(extracted_params: Dict[str, Any], all_shows_data: List[Dict[str, Any]]) -> str:
    query_show_name = extracted_params.get("show_name")
    query_date = extracted_params.get("date_query")
    query_genre = extracted_params.get("genre_query")
    relevant_shows = []
    if not all_shows_data: return "Συγγνώμη, δεν έχω διαθέσιμες πληροφορίες για παραστάσεις αυτή τη στιγμή."
    if query_show_name:
        for show in all_shows_data:
            if query_show_name.lower() in show.get("title", "").lower(): relevant_shows.append(show)
        if not relevant_shows: return f"Δεν βρέθηκε παράσταση με το όνομα '{query_show_name}'."
    else: relevant_shows = all_shows_data
    if query_date:
        temp_shows = []
        for show in relevant_shows:
            for schedule_item in show.get("schedule", []):
                if query_date.lower() in schedule_item.get("date", "").lower() or query_date.lower() in schedule_item.get("day_of_week", "").lower():
                    if show not in temp_shows: temp_shows.append(show)
                    break
        relevant_shows = temp_shows
        if not relevant_shows: return f"Δεν βρέθηκαν παραστάσεις για την ημερομηνία/μέρα '{query_date}'."
    if query_genre:
        relevant_shows = [show for show in relevant_shows if query_genre.lower() in show.get("genre", "").lower()]
        if not relevant_shows: return f"Δεν βρέθηκαν παραστάσεις του είδους '{query_genre}'."
    if not relevant_shows: return "Δεν βρέθηκαν παραστάσεις που να ταιριάζουν με τα κριτήριά σας."
    response_parts = []
    if len(relevant_shows) == 1 and query_show_name:
        show = relevant_shows[0]
        response_parts.append(f"Να οι πληροφορίες για την παράσταση '{show.get('title')}':")
        response_parts.append(f"  Περιγραφή: {show.get('description', 'Δεν υπάρχει διαθέσιμη περιγραφή.')}")
        response_parts.append(f"  Είδος: {show.get('genre', 'Άγνωστο')}")
        response_parts.append(f"  Διάρκεια: {show.get('duration_minutes', ' Άγνωστη')} λεπτά.")
        if show.get("actors"): response_parts.append(f"  Πρωταγωνιστούν: {', '.join(show.get('actors'))}")
        if show.get("schedule"):
            response_parts.append("  Πρόγραμμα:")
            for s in show.get("schedule"): response_parts.append(f"    - {s.get('day_of_week')} {s.get('date')} στις {s.get('time')}, Τιμή: {s.get('price_eur')}€ (Διαθέσιμα: {s.get('availability')})")
        if show.get("reviews"): response_parts.append(f"  Κριτικές: {' | '.join(show.get('reviews'))}")
    else:
        if query_show_name or query_date or query_genre: response_parts.append("Βρέθηκαν οι παρακάτω παραστάσεις που ταιριάζουν με τα κριτήριά σας:")
        else: response_parts.append("Αυτή τη στιγμή παίζονται οι εξής παραστάσεις:")
        for i, show in enumerate(relevant_shows):
            response_parts.append(f"\n{i+1}. {show.get('title')} ({show.get('genre', 'Άγνωστο')})")
            if not query_show_name:
                response_parts.append("   Σύντομο Πρόγραμμα:")
                for s_idx, s_item in enumerate(show.get("schedule", [])):
                    if s_idx < 2: response_parts.append(f"     - {s_item.get('day_of_week')} {s_item.get('date')} στις {s_item.get('time')} ({s_item.get('price_eur')}€)")
                    else:
                        response_parts.append("     ...και άλλες ημερομηνίες.")
                        break
                response_parts.append(f"   Για περισσότερες λεπτομέρειες, ρωτήστε με για την παράσταση '{show.get('title')}'.")
            elif len(relevant_shows) > 1 :
                response_parts.append(f"   Περιγραφή: {show.get('description', '')[:100]}...")
                response_parts.append(f"   Για πλήρεις λεπτομέρειες, διευκρινίστε τον ακριβή τίτλο.")
    return "\n".join(response_parts)

def format_theater_info_response(extracted_params: Dict[str, Any], theater_general_data: Dict[str, Any]) -> str:
    info_type = extracted_params.get("info_type", "").lower()
    if not theater_general_data: return "Δεν έχω διαθέσιμες γενικές πληροφορίες για το θέατρο αυτή τη στιγμή."
    if "διεύθυνση" in info_type or "τοποθεσία" in info_type or "πού είναι" in info_type:
        return f"Η διεύθυνση του θεάτρου '{theater_general_data.get('name', '')}' είναι: {theater_general_data.get('address', 'Δεν έχει καταχωρηθεί.')}. {theater_general_data.get('location_notes', '')}"
    elif "τηλέφωνο" in info_type or "επικοινωνία" in info_type:
        return f"Μπορείτε να επικοινωνήσετε με το θέατρο στο τηλέφωνο: {theater_general_data.get('phone', 'Δεν έχει καταχωρηθεί.')} ή στο email: {theater_general_data.get('email', 'Δεν έχει καταχωρηθεί.')}."
    elif "πάρκινγκ" in info_type:
        parking_info = theater_general_data.get('parking', {})
        if parking_info.get('available'): return f"Για το πάρκινγκ: {parking_info.get('details', 'Διατίθεται πάρκινγκ.')}"
        else: return "Δυστυχώς, δεν φαίνεται να διατίθεται ειδικός χώρος πάρκινγκ από το θέατρο."
    elif "προσβασιμότητα" in info_type or "αμεα" in info_type:
        acc_info = theater_general_data.get('accessibility', {})
        response = ["Πληροφορίες προσβασιμότητας:"]
        if acc_info.get('wheelchair_ramps'): response.append("- Υπάρχουν ράμπες για αμαξίδια.")
        if acc_info.get('accessible_seating'): response.append(f"- {acc_info.get('accessible_seating')}")
        if acc_info.get('accessible_restrooms'): response.append("- Διατίθενται προσβάσιμες τουαλέτες.")
        if acc_info.get('hearing_assistance'): response.append(f"- {acc_info.get('hearing_assistance')}")
        if acc_info.get('service_animals'): response.append(f"- {acc_info.get('service_animals')}")
        if len(response) == 1: return "Δεν έχω συγκεκριμένες πληροφορίες προσβασιμότητας."
        return "\n".join(response)
    elif "ώρες" in info_type or "ωράριο" in info_type:
        return f"Οι ώρες λειτουργίας του ταμείου είναι: {theater_general_data.get('opening_hours_box_office', 'Δεν έχουν καταχωρηθεί.')}"
    elif "ιστοσελίδα" in info_type or "website" in info_type:
        return f"Η ιστοσελίδα του θεάτρου είναι: {theater_general_data.get('website', 'Δεν έχει καταχωρηθεί.')}"
    else: return f"Καλώς ήρθατε στο θέατρο '{theater_general_data.get('name', '')}'. {theater_general_data.get('location_notes', '')} Μπορείτε να με ρωτήσετε για τις παραστάσεις, τη διεύθυνση, το πάρκινγκ κ.λπ."

print("Pydantic models, λίστα εργαλείων και συναρτήσεις μορφοποίησης απαντήσεων ορίστηκαν.")

Pydantic models, λίστα εργαλείων και συναρτήσεις μορφοποίησης απαντήσεων ορίστηκαν.


In [ ]:
# --- Αρχικοποίηση Μεταβλητών ---
llm_chat_model = None
llm_with_tools = None
chain = None # Η κύρια LangChain chain
tool_names_str = ", ".join([t.__name__ for t in tools]) # tools από Κελί 5
model_name_to_use = "gemini-2.0-flash" 
flask_app_port = 4000 # Η πόρτα που θα ακούει το Flask app

# --- Φόρτωση Δεδομένων ---
theater_full_data = load_theater_data() # load_theater_data από Κελί 5
if not theater_full_data or (not theater_full_data.get("shows") and not theater_full_data.get("theater_general_info")):
    print("ΠΡΟΕΙΔΟΠΟΙΗΣΗ NOTEBOOK: Δεν φορτώθηκαν ή είναι ελλιπή τα δεδομένα θεάτρου.")
    theater_full_data = {"theater_general_info": {}, "shows": []}

raw_theater_general_info_string = json.dumps(theater_full_data.get("theater_general_info", {}), indent=2, ensure_ascii=False)
theater_general_info_string_for_prompt = raw_theater_general_info_string.replace("{", "{{").replace("}", "}}")
raw_shows_data_string = json.dumps(theater_full_data.get("shows", []), indent=2, ensure_ascii=False)
shows_data_string_for_prompt = raw_shows_data_string.replace("{", "{{").replace("}", "}}")

# --- Αρχικοποίηση Chatbot (LLM και Chain) ---
if google_api_key: # google_api_key από Κελί 3
    try:
        print(f"Αρχικοποίηση ChatGoogleGenerativeAI με μοντέλο {model_name_to_use}...")
        llm_chat_model = ChatGoogleGenerativeAI(
            model=model_name_to_use,
            google_api_key=google_api_key,
            temperature=0.1,
            convert_system_message_to_human=True
        )
        print(f"Το ChatGoogleGenerativeAI ({model_name_to_use}) αρχικοποιήθηκε επιτυχώς.")

        print(f"Σύνδεση εργαλείων με το μοντέλο ChatGoogleGenerativeAI ({model_name_to_use})...")
        llm_with_tools = llm_chat_model.bind_tools(tools)
        print("Η σύνδεση των εργαλείων ολοκληρώθηκε.")

        system_message_content_gemini = f"""Είσαι ένα εξυπηρετικό και ακριβές AI chatbot για το θέατρο '{theater_full_data.get("theater_general_info", {}).get("name", "Το Θέατρό μας")}'.
        Ο στόχος σου είναι να βοηθάς τους χρήστες με πληροφορίες για τις παραστάσεις, να τους βοηθάς να κλείνουν εισιτήρια και να απαντάς σε γενικές ερωτήσεις για το θέατρο.
        Μιλάς άπταιστα Ελληνικά και ΠΡΕΠΕΙ πάντα να απαντάς στα Ελληνικά.
        ΠΑΝΤΑ να χρησιμοποιείς τα παρεχόμενα δεδομένα για να απαντάς στις ερωτήσεις.
        ΜΗΝ επινοείς πληροφορίες αν δεν υπάρχουν στα παρεχόμενα δεδομένα.

        Γενικές Πληροφορίες Θεάτρου:
        {theater_general_info_string_for_prompt}
        Διαθέσιμα Δεδομένα Παραστάσεων:
        {shows_data_string_for_prompt}
        Διαθέσιμα Εργαλεία/Προθέσεις: {{tool_names}}

        Οδηγίες για τη χρήση εργαλείων:
        Όταν το ερώτημα του χρήστη ταιριάζει με μία από τις παρακάτω προθέσεις, ΠΡΕΠΕΙ να καλέσεις το αντίστοιχο εργαλείο και να παρέχεις τις παραμέτρους όπως περιγράφονται στις περιγραφές των εργαλείων (οι οποίες είναι στα Ελληνικά).
        Αν καλέσεις ένα εργαλείο, η τελική σου απάντηση προς τον χρήστη πρέπει να βασίζεται στα αποτελέσματα του εργαλείου ή στις πληροφορίες που σχετίζονται με αυτό.
        Αν ο χρήστης ζητήσει κάτι που ταιριάζει σε εργαλείο, κάλεσε το εργαλείο. Μην απαντάς απευθείας αν μπορείς να χρησιμοποιήσεις εργαλείο.
        1. GetShowInfoParameters: Αν ο χρήστης ρωτά για παραστάσεις.
        2. BookTicketParameters: Αν ο χρήστης θέλει να κλείσει εισιτήρια.
        3. GetTheaterInfoParameters: Για γενικές πληροφορίες θεάτρου.
        Αν το ερώτημα του χρήστη είναι απλός χαιρετισμός, ψιλοκουβέντα ή ευχαριστίες, απάντησε φυσικά χωρίς να καλέσεις κάποιο εργαλείο.
        Ιστορικό Διαλόγου: {{chat_history}}
        """
        
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_message_content_gemini),
            ("human", "{user_input}")
        ])

        chain = prompt_template | llm_with_tools
        print(f"Η αλυσίδα (chain) με ChatGoogleGenerativeAI ({model_name_to_use}) δημιουργήθηκε.")
    except Exception as e:
        print(f"ΣΦΑΛΜΑ NOTEBOOK κατά την αρχικοποίηση του ChatGoogleGenerativeAI ή τη σύνδεση εργαλείων: {e}")
        traceback.print_exc()
else:
    print("Το Google API Key δεν είναι διαθέσιμο. Η αρχικοποίηση του LLM παραλείπεται.")

# --- Διαχείριση Ιστορικού Συνομιλίας (Global για το Flask App) ---
# Για μια εργασία, μπορούμε να χρησιμοποιήσουμε ένα απλό global dictionary για τα session histories.
# Κλειδί: session_id (αυθαίρετο string), Τιμή: λίστα από μηνύματα (HumanMessage, AIMessage)
session_chat_histories: Dict[str, List[Any]] = {}
session_collected_booking_info: Dict[str, Dict[str, Any]] = {}
MAX_HISTORY_TURNS = 7 # Μέγιστος αριθμός ζευγών user-AI μηνυμάτων στο ιστορικό

# --- Flask App ---
from flask_cors import CORS
app = Flask(__name__)
CORS(app, origins=["http://localhost:8081"])
CORS(app)



@app.route('/chat', methods=['POST'])
def chat_endpoint():
    global chain, theater_full_data, tool_names_str # Δήλωση ότι χρησιμοποιούμε τις global μεταβλητές

    if not chain:
        return jsonify({"error": "Το Chatbot (chain) δεν έχει αρχικοποιηθεί σωστά."}), 500

    try:
        data = request.json
        user_input_text = data.get('message')
        session_id = data.get('session_id', 'default_session') # Χρήση session_id για ιστορικό

        if not user_input_text:
            return jsonify({"error": "Το μήνυμα (message) είναι κενό."}), 400

        # Ανάκτηση ή δημιουργία ιστορικού για το session
        current_dialog_history = session_chat_histories.get(session_id, [])
        current_booking_info = session_collected_booking_info.get(session_id, {})
        
        print(f"Εισερχόμενο για session {session_id}: {user_input_text}")
        print(f"Τρέχον ιστορικό για session {session_id} (πριν την κλήση): {current_dialog_history}")


        chain_input = {
            "user_input": user_input_text,
            "chat_history": current_dialog_history,
            "tool_names": tool_names_str
        }
        ai_response_message = chain.invoke(chain_input)

        final_user_facing_response = ""
        extracted_info_for_ticket_json = None # Για επιστροφή JSON αν ολοκληρωθεί κράτηση

        if hasattr(ai_response_message, 'tool_calls') and ai_response_message.tool_calls and len(ai_response_message.tool_calls) > 0:
            print(f"  LLM Response ({model_name_to_use}): Εντοπίστηκε κλήση εργαλείου")
            tool_call = ai_response_message.tool_calls[0]
            recognized_intent = tool_call.get('name', 'άγνωστο_εργαλείο')
            extracted_parameters = tool_call.get('args', {})
            print(f"    -> Πρόθεση: {recognized_intent}, Παράμετροι: {extracted_parameters}")

            if ai_response_message.content and isinstance(ai_response_message.content, str) and ai_response_message.content.strip():
                final_user_facing_response = ai_response_message.content
            else:
                if recognized_intent == "GetShowInfoParameters":
                    final_user_facing_response = format_show_info_response(extracted_parameters, theater_full_data.get("shows", []))
                elif recognized_intent == "GetTheaterInfoParameters":
                    final_user_facing_response = format_theater_info_response(extracted_parameters, theater_full_data.get("theater_general_info", {}))
                elif recognized_intent == "BookTicketParameters":
                    for key, value in extracted_parameters.items():
                        if value is not None: current_booking_info[key] = value
                    
                    required_fields = ['first_name', 'last_name', 'show_name', 'num_tickets', 'date']
                    missing_fields = [f for f in required_fields if f not in current_booking_info or current_booking_info[f] is None]

                    if not missing_fields:
                        final_user_facing_response = (
                            f"Επιβεβαίωση για πιθανή κράτηση για τον/την {current_booking_info.get('first_name')} {current_booking_info.get('last_name')}:\n"
                            f"Παράσταση: {current_booking_info.get('show_name')}\nΑριθμός Εισιτηρίων: {current_booking_info.get('num_tickets')}\n"
                            f"Ημερομηνία: {current_booking_info.get('date')}\nΏρα: {current_booking_info.get('time', 'Δεν έχει οριστεί')}\n"
                            "Η κράτηση έχει προσομοιωθεί. Τα στοιχεία είναι έτοιμα."
                        )
                        extracted_info_for_ticket_json = current_booking_info.copy()
                        current_booking_info = {} # Καθαρισμός για επόμενη κράτηση στο ίδιο session
                    else:
                        final_user_facing_response = f"Για την κράτηση, παρακαλώ δώστε μου: {', '.join(missing_fields)}."
                else:
                    final_user_facing_response = f"Κατανόησα ότι πρέπει να γίνει η ενέργεια '{recognized_intent}'."
        
        elif hasattr(ai_response_message, 'content') and isinstance(ai_response_message.content, str):
            final_user_facing_response = ai_response_message.content
        else:
            final_user_facing_response = "Συγγνώμη, δεν μπόρεσα να επεξεργαστώ την απάντηση."

        # Ενημέρωση ιστορικού για το session
        current_dialog_history.append(HumanMessage(content=user_input_text))
        if isinstance(ai_response_message, AIMessage):
            current_dialog_history.append(ai_response_message)
        else:
            current_dialog_history.append(AIMessage(content=final_user_facing_response, tool_calls=getattr(ai_response_message, 'tool_calls', [])))
        
        # Περιορισμός μεγέθους ιστορικού
        if len(current_dialog_history) > MAX_HISTORY_TURNS * 2:
            current_dialog_history = current_dialog_history[-(MAX_HISTORY_TURNS * 2):]
        
        session_chat_histories[session_id] = current_dialog_history
        session_collected_booking_info[session_id] = current_booking_info # Αποθήκευση των (ενδεχομένως ενημερωμένων) στοιχείων κράτησης

        response_data = {"reply": final_user_facing_response}
        if extracted_info_for_ticket_json: # Το extracted_info_for_ticket_json ορίζεται στο Κελί 7 (στην πραγματικότητα στο Κελί 6 πλέον)
            response_data["booking_details"] = extracted_info_for_ticket_json
            # session_collected_booking_info[session_id] = {} # Καθαρισμός μετά την αποστολή

        
        json_response = json.dumps(response_data, ensure_ascii=False, indent=2) # indent=2 για όμορφη εκτύπωση αν το δείτε στο browser/postman
        return app.response_class(
            response=json_response,
            status=200,
            mimetype='application/json; charset=utf-8'
        )

    except Exception as e:
        print(f"Σφάλμα στο /chat endpoint: {e}")
        traceback.print_exc()
        return jsonify({"error": "Προέκυψε ένα εσωτερικό σφάλμα στον server."}), 500

# --- Εκκίνηση Ngrok και Flask App ---
def start_flask_app():
    # Η εκκίνηση του Flask app πρέπει να γίνει σε διαφορετικό thread αν το ngrok.connect() είναι blocking
    # ή αν το run του flask είναι blocking. Στο Kaggle, το app.run() θα μπλοκάρει το κελί.
    # Το ngrok.connect() από μόνο του συνήθως δεν μπλοκάρει αν το monitor_thread είναι False.
    print(f"Εκκίνηση Flask app στην πόρτα {flask_app_port}...")
    app.run(host='0.0.0.0', port=flask_app_port, debug=False, use_reloader=False)


if google_api_key and ngrok_authtoken and chain:
    print("Προσπάθεια εκκίνησης ngrok...")
    try:
        # Κλείσιμο τυχόν προηγούμενων tunnels για καθαρή εκκίνηση
        for t in ngrok.get_tunnels():
            ngrok.disconnect(t.public_url)
            ngrok.kill() # Διασφαλίζει ότι η διαδικασία του ngrok τερματίζεται
            print(f"Κλειστό προηγούμενο tunnel: {t.public_url}")
        
        public_url = ngrok.connect(flask_app_port).public_url # Χρήση της global μεταβλητής flask_app_port
        print(f"Το Ngrok tunnel είναι ενεργό! Public URL: {public_url}")
        print(f"Το Flask app θα ξεκινήσει σε λίγο στην πόρτα {flask_app_port}. Το API endpoint θα είναι: {public_url}/chat")

        # Για να τρέξει το Flask στο background στο Kaggle, χρειαζόμαστε threading
        # Ωστόσο, η διαχείριση threads και η εκτύπωση output στο Kaggle μπορεί να είναι περίπλοκη.
        # Μια πιο απλή προσέγγιση για demo είναι να τρέξουμε το Flask και να ξέρουμε ότι το κελί θα "κολλήσει" εδώ.
        # Η React Native εφαρμογή θα πρέπει να καλέσει το public_url που τυπώθηκε.
        
        # ΣΗΜΑΝΤΙΚΟ: Το app.run() θα μπλοκάρει την περαιτέρω εκτέλεση αυτού του κελιού.
        # Η επικοινωνία θα γίνεται μέσω του public_url από την React Native εφαρμογή.
        start_flask_app() # Αυτό θα μπλοκάρει το κελί.

    except Exception as e:
        print(f"Σφάλμα κατά την εκκίνηση του ngrok ή του Flask app: {e}")
        traceback.print_exc()
        print("Βεβαιωθείτε ότι το ngrok authtoken είναι σωστό και δεν υπάρχει άλλη παρουσία ngrok να τρέχει με τον ίδιο λογαριασμό και περιορισμούς.")
else:
    if not google_api_key: print("Το Google API Key λείπει, το Flask app δεν θα ξεκινήσει.")
    if not ngrok_authtoken: print("Το Ngrok Authtoken λείπει, το Flask app δεν θα ξεκινήσει.")
    if not chain: print("Η αλυσίδα LangChain (chain) δεν αρχικοποιήθηκε, το Flask app δεν θα ξεκινήσει.")

Αρχικοποίηση ChatGoogleGenerativeAI με μοντέλο gemini-2.0-flash...
Το ChatGoogleGenerativeAI (gemini-2.0-flash) αρχικοποιήθηκε επιτυχώς.
Σύνδεση εργαλείων με το μοντέλο ChatGoogleGenerativeAI (gemini-2.0-flash)...
Η σύνδεση των εργαλείων ολοκληρώθηκε.
Η αλυσίδα (chain) με ChatGoogleGenerativeAI (gemini-2.0-flash) δημιουργήθηκε.
Προσπάθεια εκκίνησης ngrok...
Το Ngrok tunnel είναι ενεργό! Public URL: https://0573-34-44-99-255.ngrok-free.app                  
Το Flask app θα ξεκινήσει σε λίγο στην πόρτα 4000. Το API endpoint θα είναι: https://0573-34-44-99-255.ngrok-free.app/chat
Εκκίνηση Flask app στην πόρτα 4000...
 * Serving Flask app '__main__'
 * Debug mode: off
Εισερχόμενο για session my_test_session_123: ddssd
Τρέχον ιστορικό για session my_test_session_123 (πριν την κλήση): []


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Εισερχόμενο για session my_test_session_123: Ρωμαιος και ιουλιετα
Τρέχον ιστορικό για session my_test_session_123 (πριν την κλήση): [HumanMessage(content='ddssd', additional_kwargs={}, response_metadata={}), AIMessage(content="Γεια σας! Σε τι μπορώ να σας βοηθήσω σήμερα σχετικά με το Θέατρο 'Η Αυλαία';", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--28e2ff4d-9e69-403c-8acc-088cd9b2958e-0', usage_metadata={'input_tokens': 2596, 'output_tokens': 29, 'total_tokens': 2625, 'input_token_details': {'cache_read': 0}})]


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


  LLM Response (gemini-2.0-flash): Εντοπίστηκε κλήση εργαλείου
    -> Πρόθεση: GetShowInfoParameters, Παράμετροι: {'show_name': 'Ρωμαίος και Ιουλιέτα'}
Εισερχόμενο για session my_test_session_123: 15-11
Τρέχον ιστορικό για session my_test_session_123 (πριν την κλήση): [HumanMessage(content='ddssd', additional_kwargs={}, response_metadata={}), AIMessage(content="Γεια σας! Σε τι μπορώ να σας βοηθήσω σήμερα σχετικά με το Θέατρο 'Η Αυλαία';", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--28e2ff4d-9e69-403c-8acc-088cd9b2958e-0', usage_metadata={'input_tokens': 2596, 'output_tokens': 29, 'total_tokens': 2625, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Ρωμαιος και ιουλιετα', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'GetShowInfoParameters', 'arguments': '{

...

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Εισερχόμενο για session my_test_session_123: Χάρης Ταράτσας
Τρέχον ιστορικό για session my_test_session_123 (πριν την κλήση): [HumanMessage(content='Άμλετ', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'GetShowInfoParameters', 'arguments': '{"show_name": "\\u0386\\u03bc\\u03bb\\u03b5\\u03c4"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--bd987f30-dbce-45d0-bdd0-2e8b2eb282c7-0', tool_calls=[{'name': 'GetShowInfoParameters', 'args': {'show_name': 'Άμλετ'}, 'id': '1e4f2ef2-3e40-4b02-9b43-10b182e40b18', 'type': 'tool_call'}], usage_metadata={'input_tokens': 4518, 'output_tokens': 11, 'total_tokens': 4529, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Θέλω να κλείσω ένα εισιτήριο', additional_kwargs={}, response_metadata={}), AIMessage(content='Για ποια παράσταση θα θέλατε να κ

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


  LLM Response (gemini-2.0-flash): Εντοπίστηκε κλήση εργαλείου
    -> Πρόθεση: BookTicketParameters, Παράμετροι: {'last_name': 'Ταράτσας', 'first_name': 'Χάρης', 'time': '18:00', 'date': '2025-11-15', 'num_tickets': 1.0, 'show_name': 'Άμλετ'}
Εισερχόμενο για session my_test_session_123: Επιβεβαιώνω
Τρέχον ιστορικό για session my_test_session_123 (πριν την κλήση): [HumanMessage(content='Θέλω να κλείσω ένα εισιτήριο', additional_kwargs={}, response_metadata={}), AIMessage(content='Για ποια παράσταση θα θέλατε να κλείσετε εισιτήριο;', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--85fe0360-7343-4bc1-9289-82b9fe6b3cf0-0', usage_metadata={'input_tokens': 4641, 'output_tokens': 18, 'total_tokens': 4659, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Για τον Άμλετ', additional_kwargs={}, response_metadata={}), AIMessage(content='

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
